In [ ]:
import dask_gateway

In [ ]:
gateway = dask_gateway.Gateway()

In [ ]:
if len(running_clusters := gateway.list_clusters())>0:
    cluster = gateway.connect(running_clusters[0].name)
else:
    cluster = gateway.new_cluster(conda_environment="pycon2023/pycon2023-tutorial", profile="Medium Worker")
    cluster.adapt(1,10)

In [ ]:
cluster

In [ ]:
client = cluster.get_client()
client

In [ ]:
import dask
import dask.dataframe as dd

In [ ]:
columns = [
    'YEAR', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE', 'OP_CARRIER', 
    'TAIL_NUM', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 
    'DEP_TIME', 'DEP_DELAY', 'ARR_TIME', 'ARR_DELAY', 'CANCELLED', 
    'CANCELLATION_CODE', 'DIVERTED', 'AIR_TIME', 'FLIGHTS', 'DISTANCE',
    'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 
    'LATE_AIRCRAFT_DELAY', 'DIV_ARR_DELAY'
]

In [ ]:
flights = dd.read_parquet(
    f"gcs://quansight-datasets/airline-ontime-performance/sorted/full_dataset.parquet", 
    columns=columns
)

In [ ]:
flights.head()

In [ ]:
import hvplot.dask
import hvplot.pandas

In [ ]:
hvplot.extension('bokeh')

In [ ]:
flights.groupby('FL_DATE')['DEP_DELAY'].count().hvplot()

In [ ]:
flights.hvplot.line(x='FL_DATE', y='DEP_DELAY', datashade=True)

In [ ]:
flight_subset = flights[(flights['FL_DATE']>"2018") * (flights.OP_CARRIER.isin(['AA', 'US', 'DL', 'WN']))]

In [ ]:
flight_subset.hvplot.kde('DEP_DELAY', by='OP_CARRIER', xlim=(-20, 70), width=300, subplots=True, rasterize=True)

In [ ]:
len(flights)

In [ ]:
cluster.shutdown()

In [ ]:
# close all running clusters
#for c in gateway.list_clusters():
#    gateway.stop_cluster(c.name)